In [1]:
import numpy as np
import tensorflow as tf
import pickle
from QuestionAnswers import QuestionAnswers
from nltk.tokenize import word_tokenize


In [2]:
glove_file_location='../data/glove.6B/glove.6B.300d.txt'

In [3]:
word2idx={'PAD':0}
weights=[]
with open(glove_file_location) as glove_file:
    i=0
    for index,line in enumerate(glove_file):
        line_split=line.split()
        word=line_split[0]
        word_weights=np.asarray(line_split[1:],dtype=np.float32)
        weights.append(word_weights)
        word2idx[word.lower()]=index+1
        if index>48000:
            break
        

In [4]:
Embedding_dimension=len(weights[0])
weights.insert(0,np.random.randn(Embedding_dimension))
last_word_index=len(weights)
weights.append(np.random.randn(Embedding_dimension))
word2idx['UNK']=last_word_index

In [5]:
weights=np.asarray(weights,dtype=np.float32)
Vocab_size=len(weights)

In [6]:
Max_time=50

In [7]:

glove_weights_initializer=tf.constant_initializer(weights)
encoder_inputs=tf.placeholder(dtype=tf.int32,shape=[None,Max_time])
embedding_weights=tf.get_variable(name='embedding_weights',shape=(Vocab_size,Embedding_dimension),
                                  trainable=True,initializer=glove_weights_initializer)
encoder_emb_inp=tf.nn.embedding_lookup(embedding_weights,encoder_inputs)

In [8]:
pickle_file_location='../data/squad/qa_dump'
Question_answers=[]
with open(pickle_file_location) as pickle_file:
    Question_answers=pickle.load(pickle_file)

In [9]:
len(Question_answers)

83435

In [10]:
length_questions=[]
length_answers=[]
train_questions=[]
train_answers=[]
train_size=5000
train_question_array=np.zeros((train_size,Max_time))
train_answer_array=np.zeros((train_size,Max_time))
for i,qa in enumerate(Question_answers[:5000]):
    tokenized_question=word_tokenize(qa.question)
    length_questions.append(len(tokenized_question))
    new_question=[]
    for word in tokenized_question:
        new_question.append(word2idx.get(word.lower(),word2idx['UNK']))
    while len(new_question)<Max_time:
        new_question.append(0)
    #print new_question
    train_questions.append(np.array(new_question))
    train_question_array[i]=np.array(new_question[:Max_time])
    
    tokenized_answer=word_tokenize(qa.answer)
    length_answers.append(len(tokenized_answer))
    new_answer=[]
    for word in tokenized_answer:
        new_answer.append(word2idx.get(word.lower(),word2idx['UNK']))
    while len(new_answer)<Max_time:
        new_answer.append(0)
    train_answers.append(np.array(new_answer))
    train_answer_array[i]=np.array(new_answer[:Max_time])

In [11]:
train_answers_array=np.asarray(train_answers)
sample_answer=train_answers_array[0]
print sample_answer.shape
while sample_answer.shape[0]<Max_time:
    sample_answer.append(0)
sample_answer.shape
print train_question_array.shape

(50,)
(5000, 50)


In [33]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    out=session.run(encoder_emb_inp,feed_dict={encoder_inputs:(train_question_array[:50])})
    print out.shape

(50, 50, 300)


In [12]:
lstm_size=128
inputs=tf.placeholder(dtype=tf.float32,shape=[None,None,+Embedding_dimension])
encoder_lengths=tf.placeholder(dtype=tf.int32,shape=None)
decoder_lengths=tf.placeholder(dtype=tf.int32,shape=None)
encoder_lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
encoder_output,encoder_states=tf.nn.dynamic_rnn(encoder_lstm_cell,inputs,
                                                sequence_length=encoder_lengths,dtype=tf.float32)
#decoder_lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
#decoder_output,decoder,states=tf.nn.dynamic_rnn(decoder_lstm_cell,encoder_states,
#                                                sequence_length=decoder_lengths,dtype=tf.float32)
#weights=tf.Variable(tf.random_normal([lstm_size,Embedding_dimension]))
#biases=tf.Variable(tf.random_normal([Embedding_dimension]))
#scores=tf.einsum('ijk,kl -> ijl',decoder_output,weights)+biases



In [15]:
encoder_states[0].shape

TensorShape([Dimension(None), Dimension(128)])